# Data Analysis

## View some data

In [1]:
from constants.data_constants import JORDAN_DATASET_FILEPATH, MAESTRO_DATASET_FILEPATH
from data.jordan_dataset import JordanDataset
from data.maestro_dataset import MaestroDataset

id_train_dataset = JordanDataset(
    data_dir=JORDAN_DATASET_FILEPATH,
    split="train",
    name="id_train_dataset"
)
id_test_dataset = JordanDataset(
    data_dir=JORDAN_DATASET_FILEPATH,
    split="validation",
    name="id_test_dataset"
)
ood_test_dataset = MaestroDataset(
    data_dir=MAESTRO_DATASET_FILEPATH,
    split="test",
    name="maestro_test_dataset"
)

/data/scratch/joeltjy1/conda_envs/ood-detection/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import random
import pandas as pd
from utils.process_tokens import get_readable_events

events_by_dataset = {}
for name, dataset in [("id_train", id_train_dataset), ("id_test", id_test_dataset), ("ood_test", ood_test_dataset)]:
    random_idx = random.randint(0, len(dataset) - 1)
    input_ids = dataset[random_idx]['input_ids']
    if hasattr(input_ids, 'tolist'):
        input_ids = input_ids.tolist()
    events = get_readable_events(input_ids)
    events_by_dataset[name] = events

min_events = min(len(events) for events in events_by_dataset.values())
num_rows = min(min_events, 30)

display_keys = ["misc", "onset", "dur", "inst", "pitch", "antic", "vel"]
key_mapping = {
    "misc": "special_token",
    "onset": "onset",
    "dur": "duration",
    "inst": "instrument",
    "pitch": "pitch",
    "antic": "anticipated",
    "vel": "velocity"
}

columns = pd.MultiIndex.from_product([events_by_dataset.keys(), display_keys], names=['dataset', 'key'])

table_data = []
for event_idx in range(num_rows):
    row = []
    for dataset_name in events_by_dataset.keys():
        events = events_by_dataset[dataset_name]
        event = events[event_idx]
        for display_key in display_keys:
            actual_key = key_mapping[display_key]
            value = event.get(actual_key, "")
            if hasattr(value, 'item'):
                value = value.item()
            elif hasattr(value, 'tolist'):
                value = value.tolist()
            row.append(value)
    table_data.append(row)

df = pd.DataFrame(table_data, columns=columns)
df

dataset id_train                                   id_test              ...  \
key         misc onset   dur inst pitch  antic vel    misc onset   dur  ...   
0             AR                                        AR              ...   
1                  0.0  0.14    0    G4  False               0.0  0.07  ...   
2                 0.45  0.09    0   A#4  False              0.08  0.15  ...   
3                 0.96  1.01    0    C6  False              0.13  0.59  ...   
4                 0.98   0.5    0    F3  False              0.21  0.15  ...   
5                 0.98  0.48    0    A2  False              0.35  0.14  ...   
6                 0.98  0.57    0    C4  False              0.46  0.06  ...   
7                 1.42  0.26    0    C5  False              0.58  0.13  ...   
8                 1.64  0.33    0    A5  False              0.68  0.18  ...   
9                 1.86  0.08    0    F5  False              0.77  0.16  ...   
10                2.25  1.54    0    F4  False              0.86  0.15  ...   
11                2.46   0.2    0    F5  False              0.93  0.08  ...   
12                2.71  0.18    0    F6  False              1.66  0.51  ...   
13                2.74  0.25    0    A4  False              1.76  0.53  ...   
14                3.72  0.07    0    C4  False              1.94  0.39  ...   
15          REST  4.72   0.0             False              2.06  0.84  ...   
16                4.88  0.19    0    D3  False              2.31  0.16  ...   
17                4.89  0.57    0    F4  False              2.49  0.55  ...   
18                4.89  0.32    0    D4  False              2.63  1.92  ...   
19                 4.9  0.25    0   A#1  False              2.75  0.09  ...   
20                 4.9  0.29    0   A#2  False              2.84  0.16  ...   
21                 4.9  0.44    0   A#3  False              3.02  0.23  ...   
22                4.91   0.4    0    C4  False              3.12   0.9  ...   
23                4.91  0.22    0    F2  False              3.15  0.06  ...   
24                5.19   0.1    0    D3  False              3.22  0.03  ...   
25                5.33  0.21    0    D4  False              3.31  0.07  ...   
26                5.44  0.11    0    C4  False              3.38  0.29  ...   
27                5.52  0.04    0   A#3  False              3.46  0.11  ...   
28                 5.6   0.7    0   A#2  False              3.55  0.16  ...   
29                5.71   0.6    0    F3  False              3.64  0.51  ...   

dataset                  ood_test                                    
key     pitch  antic vel     misc onset   dur inst pitch  antic vel  
0                              AR                                    
1         A#3  False                0.0  0.07    0    A2  False      
2         D#3  False               0.08  0.16    0   D#5  False      
3         D#4  False               0.08  0.14    0   D#6  False      
4         A#3  False               0.08   0.1    0    C6  False      
5         D#3  False               0.08  0.05    0    D6  False      
6          G3  False                0.1  0.14    0    C4  False      
7         A#3  False               0.28  0.07    0   A#2  False      
8         A#4  False               0.36   0.2    0    D6  False      
9          G4  False               0.37  0.18    0   A#5  False      
10        D#4  False               0.37  0.19    0    D5  False      
11        A#3  False               0.38  0.16    0    D4  False      
12         D4  False               0.58   0.1    0    C3  False      
13        A#1  False               0.68  0.24    0    C5  False      
14        A#2  False               0.68  0.21    0    C6  False      
15         D5  False               0.68  0.16    0   A#5  False      
16         D4  False               0.69  0.23    0   D#4  False      
17         F4  False               0.94   0.1    0    D3  False      
18         D3  False               1.06   0.3    0   A#5  False      
1

## Running all multivariate tests

In [3]:
from data_analysis.mardia import mardia
from data_analysis.royston import royston
from data_analysis.hz import hz
from extract_layers.pooling_functions import pool_mean_std

num_layers = 24

tests = [mardia, royston, hz]

pooling_function = pool_mean_std

## Extract layers

### Sanity check

On an actually multivariate normal distribution, all tests should return a p-value of close to 1.

In [4]:
import numpy as np

X = np.random.randn(4000, 100)
metrics = {}
for test in tests:
    metrics = metrics | test(X)

print(metrics)




  Subsampling to 1000 samples and 100 features for testing
p_value <0.001
{'skewness': np.float64(171868.379), 'kurtosis': np.float64(-2.58), 'skewness_p': 0.387, 'kurtosis_p': 0.01, 'royston_p': 0.434, 'hz_p_value': 0.001}


### Extract layers

In [5]:
from extract_layers.extract_layers_main import extract_representations, collate_fn
from constants.model_constants import JORDAN_MODEL_NAME, DEVICE
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(JORDAN_MODEL_NAME).to(DEVICE)
num_layers = 24

id_train_dataloader = DataLoader(
    id_train_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn
)

extract_representations(
    model=model,
    data=id_train_dataloader,
    pooling_function=pooling_function,
    layers=list(range(num_layers + 1)),
)

Extracting hidden states:   0%|          | 0/569 [00:00<?, ?it/s]/afs/csail.mit.edu/u/j/joeltjy1/ood-detection/src/utils/data_loading.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(ex["input_ids"], dtype=torch.long) for ex in examples]
Extracting hidden states: 100%|██████████| 569/569 [02:32<00:00,  3.73it/s]


{0: array([[-1.5858957e-04, -2.2780974e-03, -5.4641580e-03, ...,
          3.6041990e-02,  3.7796646e-02,  3.8871054e-02],
        [ 7.5466487e-05,  6.6216319e-04, -6.6342456e-03, ...,
          3.2185879e-02,  3.5957586e-02,  3.7327852e-02],
        [-1.6285133e-03, -3.2915168e-03, -2.0713529e-03, ...,
          3.7087522e-02,  3.7383165e-02,  3.8296748e-02],
        ...,
        [-1.7962824e-03,  2.3272866e-03, -6.3806227e-03, ...,
          3.4933787e-02,  3.7223965e-02,  3.6779165e-02],
        [ 1.0697738e-03, -6.2378547e-03,  3.5575293e-03, ...,
          3.6529809e-02,  3.8544346e-02,  3.8415130e-02],
        [ 6.0749738e-03, -7.5836191e-03,  1.7723180e-03, ...,
          3.4965035e-02,  3.5934646e-02,  3.8681839e-02]],
       shape=(4550, 2048), dtype=float32),
 1: array([[-0.0273591 ,  0.01079079, -0.05596501, ...,  0.09060913,
          0.09643698,  0.12677993],
        [-0.05586493,  0.00803197, -0.06769733, ...,  0.08572365,
          0.08836669,  0.12163065],
        [-0.0

In [7]:
import numpy as np
import pandas as pd
from constants.file_format import get_extract_layers_file_path
from tqdm import tqdm


metrics = []

for layer_idx in tqdm(range(num_layers+1), desc="Processing layers"):
    layer_dir = get_extract_layers_file_path(
        dataset_name="id_train_dataset",
        pooling_function_name=pooling_function.__name__,
        layer_idx=layer_idx,
    )
    layer_data = np.load(layer_dir)
    print("Processing layer", layer_idx)
    print("Layer data shape:", layer_data.shape)
    layer_metrics = {}
    for test in tests:
        print("Running test", test.__name__)
        layer_metrics = layer_metrics | test(layer_data)
    metrics.append(layer_metrics)

all_metrics = pd.DataFrame(metrics)


Processing layers:   0%|          | 0/25 [00:00<?, ?it/s]

Processing layer 0
Layer data shape: (4550, 2048)
Running test mardia
  Subsampling to 4550 samples and 100 features for testing
Running test royston
  Subsampling to 1000 samples and 100 features for testing
Running test hz
  Subsampling to 4550 samples and 100 features for testing


Processing layers:   4%|▍         | 1/25 [00:03<01:25,  3.55s/it]

p_value <0.001
Processing layer 1
Layer data shape: (4550, 2048)
Running test mardia
  Subsampling to 4550 samples and 100 features for testing
Running test royston
  Subsampling to 1000 samples and 100 features for testing
Running test hz
  Subsampling to 4550 samples and 100 features for testing


Processing layers:   8%|▊         | 2/25 [00:06<01:15,  3.27s/it]

...

p_value <0.001
Processing layer 23
Layer data shape: (4550, 2048)
Running test mardia
  Subsampling to 4550 samples and 100 features for testing
Running test royston
  Subsampling to 1000 samples and 100 features for testing
Running test hz
  Subsampling to 4550 samples and 100 features for testing


Processing layers:  96%|█████████▌| 24/25 [01:12<00:02,  2.99s/it]

p_value <0.001
Processing layer 24
Layer data shape: (4550, 2048)
Running test mardia
  Subsampling to 4550 samples and 100 features for testing
Running test royston
  Subsampling to 1000 samples and 100 features for testing
Running test hz
  Subsampling to 4550 samples and 100 features for testing


Processing layers: 100%|██████████| 25/25 [01:15<00:00,  3.02s/it]

p_value <0.001


In [8]:
all_metrics

,skewness,kurtosis,skewness_p,kurtosis_p,royston_p,hz_p_value
0,955671.223,233.142,0.001,0.001,0.001,0.001
1,966360.105,215.880,0.001,0.001,0.001,0.001
2,978932.056,135.406,0.001,0.001,0.001,0.001
3,1093661.324,149.679,0.001,0.001,0.001,0.001
4,1273953.976,193.288,0.001,0.001,0.001,0.001
5,1238129.744,191.339,0.001,0.001,0.001,0.001
6,1147395.423,170.573,0.001,0.001,0.001,0.001
7,1178409.038,171.340,0.001,0.001,0.001,0.001
8,1044300.643,136.306,0.001,0.001,0.001,0.001
9,987220.560,109.342,0.001,0.001,0.001,0.001


In [9]:

## optionally save it
all_metrics.to_csv("all_metrics.csv", index=False)